In [1]:
import pandas as pd
import requests
import cv2
import requests
from io import BytesIO
from PIL import Image
import numpy as np
df = pd.read_csv("/content/drive/MyDrive/A2_Data.csv")

In [2]:
import requests
from PIL import Image
from io import BytesIO

product_data = {}
rows_to_drop = []

for index, row in df.iterrows():
    product_id = row['Unnamed: 0']
    image_urls = eval(row['Image'])
    review_text = row['Review Text']

    # Initialize an empty list to store image data
    image_data = []

    for url in image_urls:
        try:
            response = requests.get(url)
            if response.status_code == 200:
                # Convert image data into PIL Image object
                img = Image.open(BytesIO(response.content))
                image_data.append(img)
        except Exception as e:
            print(f"Error downloading image from {url}: {e}")

    # Check if any image data is available
    if image_data:
        product_data[product_id] = (image_data, review_text)
    else:
        rows_to_drop.append(index)

# Drop rows with no image data
df.drop(rows_to_drop, inplace=True)

In [ ]:
import numpy as np
import cv2
from PIL import ImageEnhance, ImageOps

# Define a function to apply image pre-processing
def preprocess_image(img):
    # Convert PIL Image to numpy array
    img_array = np.array(img)

    # Resize image
    resized_img = cv2.resize(img_array, (new_width, new_height))  # Define new_width and new_height

    # Apply contrast enhancement
    contrast_enhancer = ImageEnhance.Contrast(Image.fromarray(resized_img))
    contrast_factor = 1.5  # Adjust contrast factor as needed
    contrast_enhanced_img = contrast_enhancer.enhance(contrast_factor)

    # Apply geometric orientation (e.g., rotation)
    rotation_angle = 30  # Rotate image by 30 degrees
    rotated_img = np.array(contrast_enhanced_img.rotate(rotation_angle))

    # Apply random flips (horizontal and vertical)
    if np.random.rand() > 0.5:
        flipped_img = cv2.flip(rotated_img, 0)  # Horizontal flip
    else:
        flipped_img = cv2.flip(rotated_img, 1)  # Vertical flip

    # Apply brightness adjustment
    brightness_enhancer = ImageEnhance.Brightness(Image.fromarray(flipped_img))
    brightness_factor = 1.2  # Adjust brightness factor as needed
    brightness_adjusted_img = brightness_enhancer.enhance(brightness_factor)

    # Apply exposure adjustment
    exposure_enhancer = ImageEnhance.Exposure(brightness_adjusted_img)
    exposure_factor = 1.2  # Adjust exposure factor as needed
    preprocessed_img = exposure_enhancer.enhance(exposure_factor)

    return preprocessed_img

# Iterate through product_data dictionary and apply pre-processing to each image
for product_id, (image_data, review_text) in product_data.items():
    preprocessed_images = []
    for img in image_data:
        preprocessed_img = preprocess_image(img)
        preprocessed_images.append(preprocessed_img)

    # Update product_data dictionary with preprocessed images
    product_data[product_id] = (preprocessed_images, review_text)


In [ ]:
+import numpy as np
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing import image

# Load MobileNet model pre-trained on ImageNet dataset
mobilenet_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Function to extract features from images using MobileNet
def extract_mobilenet_features(image_paths):
    features = []
    for img_path in image_paths:
        # Load and preprocess the image
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)

        # Extract features using MobileNet model
        features.append(mobilenet_model.predict(img_array))

    return np.array(features)

# Example usage:
# Assuming image_paths is a list containing paths to the images in the training set
image_paths = ['image1.jpg', 'image2.jpg', ...]  # Replace with actual image paths
features = extract_mobilenet_features(image_paths)

# Now 'features' contains the extracted features for each image in the training set
# You can use these features for training your classifier or any downstream task


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Function to perform text pre-processing
def preprocess_text(text):
    # Lower-casing
    text = text.lower()

    # Tokenization
    tokens = word_tokenize(text)

    # Removing punctuations and stopwords
    tokens = [token for token in tokens if token not in string.punctuation]
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Stemming
    stemmed_tokens = [stemmer.stem(token) for token in tokens]

    # Lemmatization
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return stemmed_tokens, lemmatized_tokens

# Example usage:
text = "This is an example sentence for text preprocessing."
stemmed_tokens, lemmatized_tokens = preprocess_text(text)
print("Stemmed Tokens:", stemmed_tokens)
print("Lemmatized Tokens:", lemmatized_tokens)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Sample textual reviews
reviews = [
    "The movie was great and I loved it.",
    "I didn't like the acting in the movie.",
    "The plot of the movie was too predictable."
]

# Step 1: Tokenization and Step 2: Calculate TF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(reviews)

# Step 3: Document Frequency (DF) is already calculated by TfidfVectorizer

# Step 4: Inverse Document Frequency (IDF)
idf_scores = tfidf_vectorizer.idf_

# Step 5: Calculate TF-IDF Score
tfidf_scores = tfidf_matrix.toarray()

# Print TF-IDF scores
print("TF-IDF Scores:")
for i, review in enumerate(reviews):
    print(f"Review {i+1}:")
    for j, word in enumerate(tfidf_vectorizer.get_feature_names_out()):
        print(f"{word}: {tfidf_scores[i][j]


In [ ]:
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity

# Load extracted image features
with open('image_features.pkl', 'rb') as f:
    image_features = pickle.load(f)

# Load TF-IDF scores for textual reviews
with open('tfidf_scores.pkl', 'rb') as f:
    tfidf_scores = pickle.load(f)

def find_similar_images(input_image_features, image_features, top_k=3):
    # Calculate cosine similarity between input image and all other images
    similarities = cosine_similarity(input_image_features.reshape(1, -1), image_features)

    # Get indices of top-k similar images
    similar_indices = similarities.argsort()[0][-top_k:][::-1]

    return similar_indices

# Example input: Index of the input image and its associated review
input_image_index = 0  # Assuming input_image_index is the index of the input image
input_review_index = 0  # Assuming input_review_index is the index of the input review

# Extract features of the input image
input_image_features = image_features[input_image_index]

# Find most similar images based on input image features
similar_image_indices = find_similar_images(input_image_features, image_features)

# Print the indices of the most similar images
print("Indices of top similar images:")
for idx in similar_image_indices:
    print(idx)


In [ ]:
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity

# Load TF-IDF scores for textual reviews
with open('tfidf_scores.pkl', 'rb') as f:
    tfidf_scores = pickle.load(f)

def find_similar_reviews(input_review_tfidf, all_reviews_tfidf, top_k=3):
    # Calculate cosine similarity between input review and all other reviews
    similarities = cosine_similarity(input_review_tfidf.reshape(1, -1), all_reviews_tfidf)

    # Get indices of top-k similar reviews (excluding the input review itself)
    similar_indices = similarities.argsort()[0][-top_k-1:-1][::-1]

    return similar_indices

# Example input: Index of the input review
input_review_index = 0  # Assuming input_review_index is the index of the input review

# Get TF-IDF scores for the input review
input_review_tfidf = tfidf_scores[input_review_index]

# Get TF-IDF scores for all reviews except the input review
all_reviews_tfidf = np.delete(tfidf_scores, input_review_index, axis=0)

# Find most similar reviews based on input review TF-IDF scores
similar_review_indices = find_similar_reviews(input_review_tfidf, all_reviews_tfidf)

# Print the indices of the most similar reviews
print("Indices of top similar reviews (excluding input review itself):")
for idx in similar_review_indices:
    print(idx)


In [ ]:
import pickle

# Save the indices of the most similar reviews
with open('similar_review_indices.pkl', 'wb') as f:
    pickle.dump(similar_review_indices, f)


In [ ]:
import pickle

# Load the indices of the most similar reviews
with open('similar_review_indices.pkl', 'rb') as f:
    similar_review_indices = pickle.load(f)

# Print the loaded indices
print("Loaded similar review indices:")
print(similar_review_indices)


In [ ]:
def compute_composite_similarity(text_similarity_scores, image_similarity_scores):
    """
    Computes the composite similarity score for pairs using text and image similarity scores.

    Args:
    - text_similarity_scores: Dictionary mapping text pairs to their similarity scores.
    - image_similarity_scores: Dictionary mapping image pairs to their similarity scores.

    Returns:
    - Dictionary mapping pairs to their composite similarity scores.
    """
    composite_scores = {}

    # Check if the pairs in text_similarity_scores also exist in image_similarity_scores
    for pair, text_score in text_similarity_scores.items():
        if pair in image_similarity_scores:
            # Calculate the composite similarity score
            composite_score = (text_score + image_similarity_scores[pair]) / 2
            composite_scores[pair] = composite_score

    return composite_scores

# Example similarity scores for text pairs (3a) and image pairs (3b)
text_similarity_scores = {
    ('pair1', 'text'): 0.75,
    ('pair2', 'text'): 0.60,
    ('pair3', 'text'): 0.85
}

image_similarity_scores = {
    ('pair1', 'image'): 0.90,
    ('pair2', 'image'): 0.70,
    ('pair3', 'image'): 0.80
}

# Compute composite similarity scores
composite_scores = compute_composite_similarity(text_similarity_scores, image_similarity_scores)

# Output the composite similarity scores
print("Composite Similarity Scores:")
for pair, score in composite_scores.items():
    print(pair, ":", score)


In [ ]:
# Sort the composite similarity scores in descending order
sorted_scores = sorted(composite_scores.items(), key=lambda x: x[1], reverse=True)

# Output the ranked pairs based on composite similarity score
print("Ranked Pairs based on Composite Similarity Score:")
for i, (pair, score) in enumerate(sorted_scores, start=1):
    print(f"{i}. {pair} : {score}")


In [ ]:
def compute_composite_similarity(text_similarity_scores, image_similarity_scores):
    """
    Computes the composite similarity score for pairs using text and image similarity scores.

    Args:
    - text_similarity_scores: Dictionary mapping text pairs to their similarity scores.
    - image_similarity_scores: Dictionary mapping image pairs to their similarity scores.

    Returns:
    - Dictionary mapping pairs to their composite similarity scores.
    """
    composite_scores = {}

    # Check if the pairs in text_similarity_scores also exist in image_similarity_scores
    for pair, text_score in text_similarity_scores.items():
        if pair in image_similarity_scores:
            # Calculate the composite similarity score
            composite_score = (text_score + image_similarity_scores[pair]) / 2
            composite_scores[pair] = composite_score

    return composite_scores

# Example similarity scores for text pairs (3a) and image pairs (3b)
text_similarity_scores = {
    ('pair1', 'text'): 0.75,
    ('pair2', 'text'): 0.60,
    ('pair3', 'text'): 0.85
}

image_similarity_scores = {
    ('pair1', 'image'): 0.90,
    ('pair2', 'image'): 0.70,
    ('pair3', 'image'): 0.80
}

# Compute composite similarity scores
composite_scores = compute_composite_similarity(text_similarity_scores, image_similarity_scores)

# Sort the composite similarity scores in descending order
sorted_scores = sorted(composite_scores.items(), key=lambda x: x[1], reverse=True)

# Filter out pairs consisting of an image and a review
top_ranked_pairs = [(pair, score) for pair, score in sorted_scores if pair[1] == 'text']

# Output the top-ranked (image, review) pairs along with cosine similarity scores
print("Top-ranked (image, review) pairs along with cosine similarity scores:")
for i, ((image, review), score) in enumerate(top_ranked_pairs[:10], start=1):  # Adjust the range as needed
    print(f"{i}. Image: {image}, Review: {review}, Cosine Similarity: {score}")


In [ ]:
# Calculate average composite similarity score for text-based retrieval
text_composite_scores = [score for pair, score in sorted_scores if pair[1] == 'text']
avg_text_score = sum(text_composite_scores) / len(text_composite_scores)

# Calculate average composite similarity score for image-based retrieval
image_composite_scores = [score for pair, score in sorted_scores if pair[1] == 'image']
avg_image_score = sum(image_composite_scores) / len(image_composite_scores)

# Output average composite similarity scores
print("Average Composite Similarity Score for Text-based Retrieval:", avg_text_score)
print("Average Composite Similarity Score for Image-based Retrieval:", avg_image_score)

# Compare the average scores
if avg_text_score > avg_image_score:
    print("Text-based retrieval yields a better similarity score.")
elif avg_text_score < avg_image_score:
    print("Image-based retrieval yields a better similarity score.")
else:
    print("Both retrieval techniques yield the same average similarity score.")


In [ ]:
# Sample code demonstrating challenges and potential improvements in retrieval process

# Import necessary libraries
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from PIL import Image

# Sample data
text_data = ["This is a sample text about a dog.", "A cat is sleeping on the sofa."]
image_paths = ["dog.jpg", "cat.jpg"]

# 1. Semantic Gap Challenge: Improving feature representations
# Text data processing
vectorizer = TfidfVectorizer()
text_features = vectorizer.fit_transform(text_data)

# Image data processing
def preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((224, 224))  # Resize image for ResNet50
    img = np.array(img)
    img = preprocess_input(img)
    return img

# Load pre-trained ResNet50 model for feature extraction
resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Extract image features
image_features = []
for path in image_paths:
    img = preprocess_image(path)
    img_feature = resnet_model.predict(np.expand_dims(img, axis=0))
    image_features.append(img_feature)

# 2. Data Heterogeneity Challenge: Integrating multiple modalities
# Combine text and image features
combined_features = np.concatenate((text_features.toarray(), np.array(image_features).squeeze()), axis=1)

# 3. Scalability Challenge: Efficient indexing and retrieval
# Perform similarity search using cosine similarity
similarity_matrix = cosine_similarity(combined_features)

# 4. Domain-Specific Challenges: Custom processing for specific domains
# For example, in medical imaging, you might preprocess images differently or use domain-specific models

# 5. Evaluation Metrics: Choose appropriate metrics for evaluation
# Evaluate retrieval performance using precision, recall, etc.

# 6. User Interaction and Feedback: Incorporate user feedback
# Implement interactive retrieval system with feedback mechanism

# 7. Ethical and Privacy Considerations: Address ethical concerns
# Implement privacy-preserving techniques and fairness-aware algorithms

# Example usage of similarity matrix
print("Similarity Matrix:")
print(similarity_matrix)
